In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

In [0]:
rdd1=sc.textFile("/FileStore/tables/5000_Sales_Records.csv",2)

In [0]:
rdd1.take(2)

Out[8]: ['Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit',
 'Central America and the Caribbean,Antigua and Barbuda ,Baby Food,Online,M,12/20/2013,957081544,1/11/2014,552,255.28,159.42,140914.56,87999.84,52914.72']

In [0]:
header =rdd1.first()
rdd2 = rdd1.filter(lambda row:row != header)

In [0]:
rdd2.take(2)

Out[11]: ['Central America and the Caribbean,Antigua and Barbuda ,Baby Food,Online,M,12/20/2013,957081544,1/11/2014,552,255.28,159.42,140914.56,87999.84,52914.72',
 'Central America and the Caribbean,Panama,Snacks,Offline,C,7/5/2010,301644504,7/26/2010,2167,152.58,97.44,330640.86,211152.48,119488.38']

In [0]:
rdd3=rdd2.map(lambda x:[x.split(',')[0],x.split(',')[1],x.split(',')[2],x.split(',')[3],x.split(',')[4],x.split(',')[5],x.split(',')[6],x.split(',')[7],int(x.split(',')[8]),float(x.split(',')[9]),float(x.split(',')[10]),float(x.split(',')[11]),float(x.split(',')[12]),float(x.split(',')[13])])


In [0]:
# 1. Display country wise number of orders

country_orders=rdd3.map(lambda x:(x[1],x[6])).groupBy(lambda x:x[0]).map(lambda x:(x[0],len(x[1])))
country_orders.take(5)

Out[15]: [('Antigua and Barbuda ', 26),
 ('North Korea', 24),
 ('Federated States of Micronesia', 20),
 ('Ethiopia', 26),
 ('Saint Lucia', 29)]

In [0]:
country_orders.coalesce(1).saveAsTextFile('/FileStore/tables/country_orders')

In [0]:
# 2.Display the number of units sold in each region

region_units=rdd3.map(lambda x:(x[0],x[8])).reduceByKey(lambda x,y:x+y)
print(region_units.take(10))


[('Asia', 3620036), ('Middle East and North Africa', 3013431), ('Australia and Oceania', 2111786), ('Central America and the Caribbean', 2698776), ('Europe', 6582322), ('Sub-Saharan Africa', 6642380), ('North America', 484760)]


In [0]:
region_units.coalesce(1).saveAsTextFile('/user/spark/region_units')

In [0]:
# 3.Display the 10 most recent sales. 

from datetime import datetime

dt3 = rdd3.map(lambda row: (row[:5]+ [datetime.strptime( row[5],'%m/%d/%Y')]+row[6:]))
dt3.sortBy(lambda row : row[5],ascending=False).map(lambda row : row[:5] + [row[5].strftime('%-m/%-d/%Y')] +row[6:]).take(10)

Out[19]: [['Asia',
  'Bhutan',
  'Cereal',
  'Offline',
  'M',
  '7/28/2017',
  '223854434',
  '8/25/2017',
  2356,
  205.7,
  117.11,
  484629.2,
  275911.16,
  208718.04],
 ['Sub-Saharan Africa',
  'Senegal',
  'Cosmetics',
  'Online',
  'C',
  '7/26/2017',
  '537970721',
  '8/18/2017',
  6346,
  437.2,
  263.33,
  2774471.2,
  1671092.18,
  1103379.02],
 ['Middle East and North Africa',
  'United Arab Emirates',
  'Household',
  'Online',
  'C',
  '7/26/2017',
  '419542396',
  '8/8/2017',
  773,
  668.27,
  502.54,
  516572.71,
  388463.42,
  128109.29],
 ['Australia and Oceania',
  'Australia',
  'Beverages',
  'Online',
  'L',
  '7/26/2017',
  '631485402',
  '8/12/2017',
  9418,
  47.45,
  31.79,
  446884.1,
  299398.22,
  147485.88],
 ['Sub-Saharan Africa',
  "Cote d'Ivoire",
  'Vegetables',
  'Online',
  'H',
  '7/24/2017',
  '588388097',
  '8/25/2017',
  5968,
  154.06,
  90.93,
  919430.08,
  542670.24,
  376759.84],
 ['Sub-Saharan Africa',
  'Chad',
  'Household',
  'Online',

In [0]:
# 4.Display the products with atleast 2 occurences of 'a'

a_products=rdd3.map(lambda x:x[2]).filter(lambda x:x.count('a')>=2)
print(a_products.take(5))

a_products.coalesce(1).saveAsTextFile('/user/spark/a_products')

['Personal Care', 'Personal Care', 'Personal Care', 'Personal Care', 'Personal Care']


In [0]:
#5.Display country in each region with highest units sold. 
country_sales = rdd3.map(lambda row : ((row[0] , row[1]),int(row[8])))
c_reduced = country_sales.reduceByKey(lambda a,b :a+b)
print(c_reduced.map(lambda x: (x[0][0],(x[0][1],x[1]))).reduceByKey(lambda a,b : max(a,b ,key=lambda x : x[1])).take(10))

[('Asia', ('Myanmar', 199967)), ('Australia and Oceania', ('Australia', 183909)), ('Middle East and North Africa', ('Somalia', 193065)), ('Central America and the Caribbean', ('Grenada', 205943)), ('Europe', ('Macedonia', 203078)), ('Sub-Saharan Africa', ('Equatorial Guinea', 197767)), ('North America', ('United States of America', 159519))]


In [0]:
# 6.Display the unit price and unit cost of each item in ascending order.

item_cost=rdd3.map(lambda x:(x[2],x[9],x[10])).distinct().sortBy(lambda x:x[2])
print(item_cost.take(20))

item_cost.coalesce(1).saveAsTextFile('/user/spark/item_cost')

[('Fruits', 9.33, 6.92), ('Beverages', 47.45, 31.79), ('Clothes', 109.28, 35.84), ('Personal Care', 81.73, 56.67), ('Vegetables', 154.06, 90.93), ('Snacks', 152.58, 97.44), ('Cereal', 205.7, 117.11), ('Baby Food', 255.28, 159.42), ('Cosmetics', 437.2, 263.33), ('Meat', 421.89, 364.69), ('Household', 668.27, 502.54), ('Office Supplies', 651.21, 524.96)]


In [0]:
# 7.Display the number of sales yearwise.

yearwise_sales=rdd3.map(lambda x:(str(x[5])[:4],x[6])).groupBy(lambda x:x[0]).map(lambda x:(x[0],len(x[1])))
print(yearwise_sales.take(10))

yearwise_sales.coalesce(1).saveAsTextFile('/user/spark/yearwise_sales')

[('12/2', 128), ('9/12', 17), ('5/13', 12), ('9/25', 23), ('5/12', 9), ('7/31', 11), ('8/13', 10), ('10/3', 43), ('3/13', 13), ('4/16', 14)]


In [0]:

# 8.Display the number of orders for each item.

item_orders=rdd3.map(lambda x:(x[2],x[6])).groupBy(lambda x:x[0]).map(lambda x:(x[0],len(x[1])))
print(item_orders.take(15))

item_orders.coalesce(1).saveAsTextFile('/user/spark/item_orders')

[('Baby Food', 445), ('Snacks', 398), ('Cereal', 385), ('Clothes', 386), ('Cosmetics', 424), ('Fruits', 447), ('Beverages', 447), ('Personal Care', 415), ('Office Supplies', 420), ('Meat', 399), ('Vegetables', 410), ('Household', 424)]


In [0]:
import subprocess

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-4306414689821425> in <cell line: 1>()
----> 1 subprocess.run("dbfs","-rm","/user","/spark","a_products")

/usr/lib/python3.9/subprocess.py in run(input, capture_output, timeout, check, *popenargs, **kwargs)
    503         kwargs['stderr'] = PIPE
    504 
--> 505     with Popen(*popenargs, **kwargs) as process:
    506         try:
    507             stdout, stderr = process.communicate(input, timeout=timeout)

/usr/lib/python3.9/subprocess.py in __init__(self, args, bufsize, executable, stdin, stdout, stderr, preexec_fn, close_fds, shell, cwd, env, universal_newlines, startupinfo, creationflags, restore_signals, start_new_session, pass_fds, user, group, extra_groups, encoding, errors, text, umask)
    776             bufsize = -1  # Restore default
    777         if not isinstance(bufsize, int):
--> 778             raise Typ